In [1]:
import os
import pandas as pd
from modules.data_pipeline import read_path, get_path, export_file

export_file.export_to_csv(99, 'igv')
# read_path.read(97, 'igv')

Before filter: (12800, 28)
After filter (5723, 21)
Now processing ---> ('CK', 'ck20240408', 'V001_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (6377, 21)
Now processing ---> ('CK', 'ck20240408', 'V002_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (3258, 21)
Now processing ---> ('CK', 'ck20240408', 'V003_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (3563, 21)
Now processing ---> ('CK', 'ck20240408', 'V004_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (3956, 21)
Now processing ---> ('CK', 'ck20240408', 'V005_IGVData_KPI.csv')
Before filter: (11802, 28)
After filter (4620, 21)
Now processing ---> ('CK', 'ck20240408', 'V006_IGVData_KPI.csv')
Before filter: (12781, 28)
After filter (3228, 21)
Now processing ---> ('CK', 'ck20240408', 'V008_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (2800, 21)
Now processing ---> ('CK', 'ck20240408', 'V009_IGVData_KPI.csv')
Before filter: (12553, 28)
After filter (3904, 21)
Now processing ---> (

KeyError: 'distance(m)'

In [ ]:
import pandas as pd

def __strip_str__(df=pd.DataFrame):
    """
    Strip leading and trailing whitespaces from column names and string columns.

    Parameters:
    DataFrame (pd.DataFrame): Input DataFrame. 

    Returns:
    pd.DataFrame: DataFrame with leading and trailing whitespaces stripped from column names and string columns.
    """
    df.columns = df.columns.str.strip()
    
    # Select columns of type 'object'
    obj_cols = df.select_dtypes(['object']).columns
    
    # Strip leading and trailing whitespaces from string columns
    df[obj_cols] = df[obj_cols].apply(lambda x: x.str.strip())

    return df

# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\TJ20240409-0410kpi\A023_IGVData_KPI.csv"
# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\\tpy20240409\\tpy06_IGVData.csv"
# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\\WH20240408\宝旺8号\TD002_IGVData_KPI.csv"
# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\TS20240411\\556_IGVData.csv"
p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\dl_20240408\dlqt_2_IGVData_KPI.csv"

df = pd.read_csv(p)
df = __strip_str__(df)

display(df['vesselVisitID'].value_counts().index.tolist(), df.shape)

data = df.copy()
val_to_drop = ['None', '']
data = data[data['vesselVisitID'].isin(val_to_drop)==False]

display(data['vesselVisitID'].value_counts().index.tolist(), data.shape)

In [ ]:
p = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TJ.csv'
p = os.path.abspath(p)

df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)
df.columns

In [ ]:
def lmd_get_current_task_tag(current_task=str, mission_type=str, location_ref=str):
    if (current_task != 'DSCH') & (current_task != 'LOAD'):
        if ('YARD' in location_ref and mission_type=='DELIVER') or ('QC' in location_ref and mission_type=='RECEIVE'):
            return 'DSCH'
        elif('QC' in location_ref and mission_type=='DELIVER') or ('YARD' in location_ref and mission_type=='RECEIVE'):
            return 'LOAD'      
    else:
        return current_task
    
df['current_task_tag'] = df.apply(
                    lambda x: lmd_get_current_task_tag(x['current_task'], x['mission_type'], x['location_ref']),
                    axis=1)


df.columns

# Test all weekly file

In [1]:
import os
import pandas as pd
import numpy as np

def get_all_weekly_file():
    root = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99'
    pathlist = []
    for item in os.listdir(root):
        p = os.path.join(root, item)
        if os.path.isfile(p):
            pathlist.append(p)
    return pathlist

def lmd_rtg_operation(location_ref=str, task_stage=str):
    if ('YARD' in location_ref) & (task_stage=='Waiting for Operation'):
        return 'Waiting for Operation'
    else:
        return np.NaN

li = get_all_weekly_file()

In [19]:
for p in li:
    print(p)

    df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)

    time_col = 'local_time'
    df[time_col] = pd.to_datetime(df[time_col], format='%Y-%m-%d %H:%M:%S')    
    df['time_diff'] = (df[time_col] - df[time_col].shift(1)).dt.total_seconds().fillna(3)
    
    if 'Cycle Tag' in df.columns:
        for cycle, data in df.groupby('Cycle Tag'):
            data['container_tag'] = data['container1_type'].astype(str) + data['container2_type'].astype(str)

            rtg_ind = data[
                (data['task_stage']=='Waiting for Operation') & data['location_ref'].str.contains('YARD', na=False)
                ].index.to_list()
            
            rtg_data = data[data.index.isin(rtg_ind)]
            n_mission= len(rtg_data['missionID'].value_counts().index.tolist())
            cntr_tag = data['container_tag'].value_counts().index.tolist()[0]
            
            if n_mission>=2:
                container_tag = '2020'
            else:
                container_tag = cntr_tag

            data_ind = data.index
            df.loc[data_ind, "container_tag"] = container_tag
            
    print()

c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_CK.csv

c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_DL.csv

c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_ICA.csv


C:\Users\westwell\AppData\Local\Temp\ipykernel_21864\1971800734.py:4: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)



c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TJ.csv

c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TPY.csv

c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TS.csv


C:\Users\westwell\AppData\Local\Temp\ipykernel_21864\1971800734.py:4: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)



c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_WH.csv

c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_YH.csv



In [3]:
df.columns


Index(['vehicle_id', 'position_x', 'position_y', 'position_z', 'heading',
       'speed', 'speed_command', 'mileage_distance', 'soc', 'local_time',
       'chassis_mode', 'estop', 'charge_status', 'task_state_running',
       'task_state_estop', 'task_state_lock', 'task_stage', 'current_task',
       'error_code', 'target_location', 'vesselVisitID', 'mission_type',
       'container1_type', 'container2_type', 'vehicle_mode', 'missionID',
       'lane_change_state', 'location_ref', 'location_tag', 'current_task_tag',
       'cycle', 'mission_type_change', 'Cycle Tag', 'time_diff'],
      dtype='object')

In [4]:
df['task_stage'].value_counts().index.tolist()

['Navigation', 'Alignment', 'Waiting for Operation', 'Operation Finished']

In [ ]:
path = os.path.abspath('c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TPY.csv')

d = pd.read_csv(path, encoding='utf-8-sig')
print(d['vesselVisitID'].value_counts())

In [ ]:
for cycle, data in d.groupby('Cycle Tag'):
    if len(data)>100:
        print(cycle)
        col = ['container1_type', 'container2_type']
        print(data[col].value_counts())
   
    print()